In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
import gc
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.utils.class_weight import compute_sample_weight

In [2]:
df_sub=pd.read_csv("/kaggle/input/playground-series-s5e6/sample_submission.csv")
df_train=pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
df_original = pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")
df_test=pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")

In [3]:
df_train = pd.concat([df_train, df_original], axis=0, ignore_index=True)
df_train = df_train.drop(columns=['id'])
df_test = df_test.drop(columns=['id'])

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850000 entries, 0 to 849999
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Temparature      850000 non-null  int64 
 1   Humidity         850000 non-null  int64 
 2   Moisture         850000 non-null  int64 
 3   Soil Type        850000 non-null  object
 4   Crop Type        850000 non-null  object
 5   Nitrogen         850000 non-null  int64 
 6   Potassium        850000 non-null  int64 
 7   Phosphorous      850000 non-null  int64 
 8   Fertilizer Name  850000 non-null  object
dtypes: int64(6), object(3)
memory usage: 58.4+ MB


In [5]:
categorical_columns = df_train.select_dtypes(include=['object']).columns
unique_values = {col: df_train[col].nunique() for col in categorical_columns}
for col, unique_count in unique_values.items():
    print(f"{col}: {unique_count} unique values")
    
gc.collect()

Soil Type: 5 unique values
Crop Type: 11 unique values
Fertilizer Name: 7 unique values


0

In [6]:
categorical_columns = df_test.select_dtypes(include=['object']).columns
unique_values = {col: df_test[col].nunique() for col in categorical_columns}
for col, unique_count in unique_values.items():
    print(f"{col}: {unique_count} unique values")
    
gc.collect()

Soil Type: 5 unique values
Crop Type: 11 unique values


0

In [7]:
missing_train = df_train.isna().mean() * 100
missing_test = df_test.isna().mean() * 100

print("Columns in df_train with more than 10% missing values:")
print(missing_train[missing_train > 0])

print("\nColumns in df_test with more than 10% missing values:")
print(missing_test[missing_test > 0])

Columns in df_train with more than 10% missing values:
Series([], dtype: float64)

Columns in df_test with more than 10% missing values:
Series([], dtype: float64)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
import category_encoders as ce

missing_threshold = 0.95

high_missing_columns = df_train.columns[df_train.isnull().mean() > missing_threshold]

df_train = df_train.drop(columns=high_missing_columns)
df_test = df_test.drop(columns=high_missing_columns)

for column in df_train.columns:
    if df_train[column].isnull().any():      
        if df_train[column].dtype == 'object':
            mode_value = df_train[column].mode()[0]
            df_train[column].fillna(mode_value, inplace=True)
            df_test[column].fillna(mode_value, inplace=True)     
        else:
            median_value = df_train[column].median()
            df_train[column].fillna(median_value, inplace=True)
            df_test[column].fillna(median_value, inplace=True)


df_train.columns


Index(['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type',
       'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name'],
      dtype='object')

In [9]:
cat_cols_train = df_train.select_dtypes(include=['object']).columns
cat_cols_train = cat_cols_train[cat_cols_train != 'Fertilizer Name']
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

df_train[cat_cols_train] = ordinal_encoder.fit_transform(df_train[cat_cols_train].astype(str))
df_test[cat_cols_train] = ordinal_encoder.transform(df_test[cat_cols_train].astype(str))

cat_features = ['Soil Type','Crop Type']
df_train['Soil Type'] = df_train['Soil Type'].astype('category').cat.codes
df_train['Crop Type'] = df_train['Crop Type'].astype('category').cat.codes

df_test['Soil Type'] = df_test['Soil Type'].astype('category').cat.codes
df_test['Crop Type'] = df_test['Crop Type'].astype('category').cat.codes



In [10]:
df_train.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,37,70,36,1,8,36,4,5,28-28
1,27,69,65,4,4,30,6,18,28-28
2,29,63,32,4,4,24,12,16,17-17-17
3,35,62,54,4,0,39,12,4,10-26-26
4,35,58,43,3,6,37,2,16,DAP


In [11]:
df_test.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,31,70,52,4,10,34,11,24
1,27,62,45,3,8,30,14,15
2,28,72,28,1,2,14,15,4
3,37,53,57,0,2,18,17,36
4,31,55,32,3,7,13,19,14


In [12]:
le = LabelEncoder()
df_train['Fertilizer Name'] = le.fit_transform(df_train['Fertilizer Name'])

y = df_train['Fertilizer Name'] 
X = df_train.drop(['Fertilizer Name'],axis=1)

In [13]:
# KFold setup
FOLDS = 5
kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

# Arrays to store predictions
oof = np.zeros((len(df_train), len(np.unique(y))))
pred = np.zeros((len(df_test), len(np.unique(y))))
logloss = []

# Start CV loop
for i, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"\n{'#'*10} Fold {i+1} {'#'*10}")
    
    x_train = X.iloc[train_idx].copy()
    y_train = y.iloc[train_idx]
    x_valid = X.iloc[valid_idx].copy()
    y_valid = y.iloc[valid_idx]
    x_test = df_test.copy()

    #weights = compute_sample_weight(class_weight='balanced', y=y_train)
    # 将数据转换为 XGBoost 的 DMatrix 格式
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dvalid = xgb.DMatrix(x_valid, label=y_valid)
    dtest = xgb.DMatrix(x_test)

    # XGBoost 参数
    params = {
        'objective': 'multi:softprob',  # 多分类概率输出
        'num_class': len(np.unique(y)),  # 类别数
        'max_depth': 16,
        'learning_rate': 0.03,
        'min_child_weight' : 2,
        'alpha': 0.8, 
        'reg_lambda': 4.0, 
        'colsample_bytree': 0.3,
        'subsample': 0.8,
        'max_bin': 128,
        'colsample_bytree': 0.3, 
        'colsample_bylevel': 1,  
        'colsample_bynode': 1,  
        'tree_method': 'hist',  
        'random_state': 42,
        'eval_metric': 'mlogloss',
        #'tree_method': 'hist',  
        #'device': 'cpu'                 
    }

    # 训练模型
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=5000,
        evals=[(dvalid, 'valid')],
        early_stopping_rounds=50,
        verbose_eval=200
    )

    # Predict OOF and test
    oof[valid_idx] = model.predict(dvalid)
    pred += model.predict(dtest)

    log_loss_value = log_loss(y_valid, oof[valid_idx])
    print(f"Fold {i+1} log_loss: {log_loss_value:.4f}")
    logloss.append(log_loss_value)

# Average test predictions
pred /= FOLDS
log_loss_value = np.mean(logloss)

print(f"\nFinal CV log_loss: {log_loss_value:.4f}")


########## Fold 1 ##########
[0]	valid-mlogloss:1.94564
[200]	valid-mlogloss:1.91877
[400]	valid-mlogloss:1.90653
[600]	valid-mlogloss:1.89908
[800]	valid-mlogloss:1.89429
[1000]	valid-mlogloss:1.89140
[1200]	valid-mlogloss:1.88970
[1400]	valid-mlogloss:1.88903
[1577]	valid-mlogloss:1.88897
Fold 1 log_loss: 1.8890

########## Fold 2 ##########
[0]	valid-mlogloss:1.94565
[200]	valid-mlogloss:1.91881
[400]	valid-mlogloss:1.90671
[600]	valid-mlogloss:1.89940
[800]	valid-mlogloss:1.89464
[1000]	valid-mlogloss:1.89181
[1200]	valid-mlogloss:1.89018
[1400]	valid-mlogloss:1.88960
[1517]	valid-mlogloss:1.88953
Fold 2 log_loss: 1.8895

########## Fold 3 ##########
[0]	valid-mlogloss:1.94564
[200]	valid-mlogloss:1.91872
[400]	valid-mlogloss:1.90645
[600]	valid-mlogloss:1.89906
[800]	valid-mlogloss:1.89430
[1000]	valid-mlogloss:1.89159
[1200]	valid-mlogloss:1.88995
[1400]	valid-mlogloss:1.88934
[1516]	valid-mlogloss:1.88932
Fold 3 log_loss: 1.8893

########## Fold 4 ##########
[0]	valid-mlogloss:

In [14]:
top_3_preds = np.argsort(pred, axis=1)[:, -3:][:, ::-1]  
actual = [[label] for label in y]

def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        p = p[:k]
        score = 0.0
        hits = 0
        seen = set()
        for i, pred in enumerate(p):
            if pred in a and pred not in seen:
                hits += 1
                score += hits / (i + 1.0)
                seen.add(pred)
        return score / min(len(a), k)
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])
map3_score = mapk(actual, top_3_preds)
print(f"✅ MAP@3 Score: {map3_score:.5f}")

✅ MAP@3 Score: 0.26695


In [15]:
top_3_preds = np.argsort(pred, axis=1)[:, -3:][:, ::-1]
top_3_labels = le.inverse_transform(top_3_preds.ravel()).reshape(top_3_preds.shape)
submission = pd.DataFrame({
    'id': df_sub['id'],
    'Fertilizer Name': [' '.join(row) for row in top_3_labels]
})
submission.to_csv('submission.csv', index=False)
print("✅ Submission file saved as 'submission.csv'")


✅ Submission file saved as 'submission.csv'
